## THE ONE IN WHICH I USE NYTIMES API

So basically I realized that the task on hand could be done much more quickly by using the api provided by New York Times themselves. Also the number of articles retrieved is higher making LDA run much better.
Only catch is that the __API does not provide us with the actual articles__ hence we __use the newsplease library to extract texts using the urls provided__

In [1]:
#This is my nytimes developer API key required to make the requests.
API_KEY="3EhyoUDWeGwW0vTGCY931FOgXOHCGBAj"

In [2]:
import time

In [3]:
import requests

In [43]:
#taking the input of the date
date_str=input('Please enter a date in mm-dd-yyyy format')

Please enter a date in mm-dd-yyyy format12-22-2016


In [44]:
#converting it into a form required to make request yyyymmdd
print(date_str)
date_trimmed=date_str[6:]+date_str[0:2]+date_str[3:5]
print(date_trimmed)

12-22-2016
20161222


In [46]:
PREFIX="http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date="
CENTER="&end_date="
SUFFIX="&api-key="

In [74]:
#making the first url
#this will help us to get the total number of documents
url=PREFIX+date_trimmed+CENTER+date_trimmed+"&offset=1"+SUFFIX+API_KEY
print(url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20161222&end_date=20161222&offset=1&api-key=3EhyoUDWeGwW0vTGCY931FOgXOHCGBAj


In [56]:
#retrieving the json containing the inforamtion
sample_json=requests.get(url).json()

In [57]:
#having a look at the json
type(sample_json)

dict

In [58]:
#looking at keys of the dictionary
sample_json.keys()

dict_keys(['status', 'copyright', 'response'])

In [60]:
#looking at the response key value
response=sample_json['response']

In [61]:
#looking at the response field
type(response)

dict

In [62]:
response.keys()

dict_keys(['docs', 'meta'])

In [67]:
#looking at docs
docs = response['docs']
print(docs)

#looking at meta
meta=response['meta']
print(meta)

[{'web_url': 'https://www.nytimes.com/slideshow/2016/12/22/blogs/choosing-the-new-york-times-pictures-of-the-year.html', 'snippet': 'How do you sort through 180,000 images to find a handful that sum up the year? Jeffrey Henson Scales explains.', 'lead_paragraph': 'How do you sort through 180,000 images to find a handful that sum up the year? Jeffrey Henson Scales explains.', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'wide', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2016/12/21/blogs/21-lens-yip-slide-DM6V/21-lens-yip-slide-DM6V-thumbWide.jpg', 'height': 126, 'width': 190, 'legacy': {'wide': 'images/2016/12/21/blogs/21-lens-yip-slide-DM6V/21-lens-yip-slide-DM6V-thumbWide.jpg', 'wideheight': '126', 'widewidth': '190'}, 'subType': 'wide', 'crop_name': None}, {'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2016/12/21/blogs/21-lens-yip-slide-DM6V/21-lens-yip-slide-DM6V-articleLarg

In [68]:
#looking at docs
type(docs)

list

In [71]:
#looking at a single element
type(docs[0])

dict

In [73]:
sample_docs=docs[0]
sample_docs.keys()


dict_keys(['web_url', 'snippet', 'lead_paragraph', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'slideshow_credits', 'score'])

## The main part
We need to use a page arguement in the query as well to get results as each page only displays 10 documents

In [168]:
date_str=input('Please enter a date in mm-dd-yyyy format\n')
date_trimmed=date_str[6:]+date_str[0:2]+date_str[3:5]
DATE=date_trimmed

PREFIX="http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date="
CENTER="&end_date="
SUFFIX="&api-key="

Please enter a date in mm-dd-yyyy format
02-14-1997


In [169]:
#getting the json file by making request to the api
response_json=requests.get(url).json()

In [170]:
#finding total number of documents
num_docs=response_json['response']['meta']['hits']
print(num_docs)

#adding the documents on first page in a newly created list of all document
docs=response_json['response']['docs']

234


In [171]:
#since each page contains 10 queries
if num_docs%10 is 0:
    adjuster=1
else:
    adjuster=0

total_pages = int(num_docs/10)+1-adjuster

In [172]:
for page in range(1,total_pages):
    #creating the url, basically page arguement allows us to get different documents
    #page=0 gives first 10, page=1 gives next 10 and so on
    url=PREFIX+DATE+CENTER+DATE+"&page="+str(page)+SUFFIX+API_KEY
    response_json=requests.get(url).json()
    
    #Okay so nytimes allows only 10 queries in a minute(and only 4000 in a day)
    #so let's wait a minute before making the same query again
    if not 'response' in response_json.keys():
        print("Waiting for 1 minute before making another request")
        time.sleep(60)
        response_json=requests.get(url).json()        
    
    #adding these documents to our list as well
    new_docs = response_json['response']['docs']
    docs.extend(new_docs)
    

Waiting for 1 minute before making another request
Waiting for 1 minute before making another request


In [180]:
docs[233].keys()

dict_keys(['web_url', 'snippet', 'lead_paragraph', 'print_page', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'score', 'uri'])

In [179]:
#now we will create a dictionary to check varios things before moving forward
import pickle

with open('filename.pickle', 'wb') as handle:
    pickle.dump(docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle

with open('filename.pickle', 'rb') as handle:
    docs = pickle.load(handle)

In [6]:
#creating a list of web-urls
web_urls = []

for doc in docs:
    web_urls.append(doc['web_url'])

## COULD BE DONE

you can run the below cell to not include paid death notices.I have not done so as wanted to see how much 
portion of NYT is filled with these type of ads.

In [ ]:

#put all the urls not related to "Paid Death Notices" in the list of url's to be downloaded
# web_urls = [doc['web_url'] for doc in docs if not doc['section_name'] == "Paid Death Notices"]

In [11]:
#Now please note that the api of nytimes does not provide us with text
#but we can use news-please for that since we have extrated the urls
print(len(web_urls))
print(web_urls[45])

234
https://www.nytimes.com/1997/02/14/arts/new-video-releases-630004.html


In [8]:
from newsplease import NewsPlease

In [12]:
articles=[]

for i in range(len(docs)):
    try:
        article=NewsPlease.from_url(web_urls[i])
        articles.append(article)
        print("Success in article {}".format(i))
    except:
        print("Encountered HTTPError in one of the pages moving on")
        print(web_urls[i])

Success in article 0
Success in article 1
Success in article 2
Encountered HTTPError in one of the pages moving on
https://www.nytimes.com/1997/02/14/us/space-repair-job-begins.html
Success in article 4
Success in article 5
Success in article 6
Success in article 7
Success in article 8
Success in article 9
Success in article 10
Success in article 11
Success in article 12
Success in article 13
Success in article 14
Success in article 15
Success in article 16
Success in article 17
Success in article 18
Success in article 19
Success in article 20
Success in article 21
Success in article 22
Success in article 23
Success in article 24
Success in article 25
Success in article 26
Success in article 27
Success in article 28
Success in article 29
Success in article 30
Success in article 31
Success in article 32
Success in article 33
Success in article 34
Success in article 35
Success in article 36
Success in article 37
Success in article 38
Success in article 39
Success in article 40
Success in

In [13]:
len(articles)

232

In [15]:
articles_in_dic_format = [article.get_dict() for article in articles]

In [44]:
articles_in_dic_format[2]

{'authors': [],
 'date_download': datetime.datetime(2019, 3, 17, 6, 56, 41),
 'date_modify': None,
 'date_publish': datetime.datetime(1997, 2, 14, 0, 0),
 'description': 'Dance Theater of Harlem and American Guild of Musical Artists agree on two-year contract for dancers, ending 19-day strike (S)',
 'filename': 'https%3A%2F%2Fwww.nytimes.com%2F1997%2F02%2F14%2Farts%2Fharlem-dance-contract.html.json',
 'image_url': 'https://static01.nyt.com/images/icons/t_logo_291_black.png',
 'language': 'en',
 'localpath': None,
 'source_domain': 'www.nytimes.com',
 'text': "The Dance Theater of Harlem and the American Guild of Musical Artists, which represents the company's dancers, announced yesterday that they had reached an agreement on a new two-year contract, ending a 19-day strike. Under the contract, the 32 dancers will receive a 5 percent raise in each year. Under the old contract, which expired Sept. 30, the wage scale ran from $269 a week for apprentices to $767 a week for principal dancers

In [19]:
docs[0]

{'web_url': 'https://www.nytimes.com/1997/02/14/classified/paid-notice-deaths-kraus-george-e.html',
 'snippet': 'KRAUS-George E., 88, of Scottsdale, AZ, formerly of New York, died February 13. Beloved husband of Bobbi, cherished father and father-in-law to Marlene and Steve Morris of New York and Leslie Kraus and Susan Rosenthal Kraus of Minnesota. Loving gr...',
 'lead_paragraph': 'KRAUS-George E., 88, of Scottsdale, AZ, formerly of New York, died February 13. Beloved husband of Bobbi, cherished father and father-in-law to Marlene and Steve Morris of New York and Leslie Kraus and Susan Rosenthal Kraus of Minnesota. Loving grandfather of David and Gary Morris and Erica Rosenthal Kraus. His goodness, wisdom and sense of humor will be forever appreciated. Contributions may be made to the charity of your choice.',
 'print_page': '7',
 'blog': {},
 'source': 'The New York Times',
 'multimedia': [],
 'headline': {'main': 'Paid Notice: Deaths  KRAUS, GEORGE E.',
  'kicker': None,
  'content_

In [43]:
#counting the number of paid death notices as don't think they are
#really important
num=0
str_match = docs[0]['section_name']
print(str_match)

for doc in docs:
    if doc['section_name']==str_match:
        num+=1
print(num)

Paid Death Notices
42


In [38]:
for i,doc in enumerate(docs):
    print(i,doc['section_name'])

0 Paid Death Notices
1 Paid Death Notices
2 Arts
3 Front Page
4 Paid Death Notices
5 Arts
6 Sports
7 Opinion
8 World
9 Sports
10 Arts
11 Arts
12 New York and Region
13 Arts
14 Opinion
15 Paid Death Notices
16 Paid Death Notices
17 Front Page; U.S.
18 Sports
19 New York and Region
20 New York and Region
21 Opinion
22 Opinion
23 New York and Region
24 Business
25 Opinion
26 Front Page; U.S.
27 World; Washington
28 Arts; Theater
29 Business
30 New York and Region
31 Business
32 Business
33 Sports
34 Business
35 U.S.
36 Movies; Arts
37 Opinion
38 Front Page; New York and Region
39 Paid Death Notices
40 Paid Death Notices
41 Arts
42 Arts
43 New York and Region
44 Business
45 Arts
46 Paid Death Notices
47 Sports
48 Opinion
49 Sports
50 Business
51 Business
52 Business
53 Movies; Arts
54 Style; Washington
55 Paid Death Notices
56 New York and Region
57 U.S.
58 New York and Region
59 Paid Death Notices
60 Sports
61 Paid Death Notices
62 World
63 New York and Region
64 Opinion
65 New York and R

## Let's get started with LDA data creation

In [50]:
#looking at the keys to select important ones
articles_in_dic_format[0].keys()

dict_keys(['authors', 'date_download', 'date_modify', 'date_publish', 'description', 'filename', 'image_url', 'language', 'localpath', 'source_domain', 'text', 'title', 'title_page', 'title_rss', 'url'])

In [63]:
sample_article=articles_in_dic_format[4]

In [68]:
print(sample_article['description'])
print(sample_article['authors'])
print("\n\n"+sample_article['text'])
print("\n\n"+sample_article['title'])

List of some upcoming entertainment events for children in New York
['Laurel Graeber']


'Let's Be Friends'
'Sesame Street Live'
The Theater at Madison Square Garden
Seventh Avenue and 33d Street
(212) 465-6741
Recommended ages: 1 through 6
Through Feb. 23
Does Madison Square Garden have earthquake insurance? This question comes to mind when the pitter-patter of little feet becomes more like the thump-thump of a many-footed monster.
Monsters, in fact, are all over the stage at the latest ''Sesame Street Live'' presentation, where the furry Muppets encourage young audiences to act out lyrics like, ''If you're grouchy and you know it stomp your feet.'' But with this welcome plunge into naughtiness, the show also offers lots of advice on friendship, from how to say ''Hello, my friend'' in Spanish to how to pitch in and help a buddy clean up his room.
The cast includes all the usual suspects, from Hoots the Owl to the Honkers, but some of the more memorable appearances are by creatures who

In [55]:
docs[0].keys()

dict_keys(['web_url', 'snippet', 'lead_paragraph', 'print_page', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'type_of_material', '_id', 'word_count', 'score'])

In [58]:
for doc in docs:
    print(doc['headline'])

{'main': 'Paid Notice: Deaths  KRAUS, GEORGE E.', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'Paid Notice: Deaths  OWEN, IVAN G., SR.', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'Harlem Dance Contract', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'Space Repair Job Begins', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'Paid Notice: Deaths  MAIER, EVA', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'For Children', 'kicker': 'For Children', 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}
{'main': 'Rangers, Gretzky Are Unable To Produce', 'kicker': 'HOCKEY', 'content_kicker': None, 'print_headline': None, 'name'

In [67]:
docs[5]

{'web_url': 'https://www.nytimes.com/1997/02/14/arts/for-children.html',
 'snippet': "'The African Drum'  Shadow Box Theater  Y.W.C.A. of Brooklyn  30 Third Avenue, at Atlantic Avenue  Boerum Hill  (212) 724-0677  Recommended ages: 2 to 10  Tomorrow and March 1      There isn't much in Brooklyn that resembles the Serengeti Plains. ...",
 'lead_paragraph': "'The African Drum' Shadow Box Theater Y.W.C.A. of Brooklyn 30 Third Avenue, at Atlantic Avenue Boerum Hill (212) 724-0677 Recommended ages: 2 to 10 Tomorrow and March 1 There isn't much in Brooklyn that resembles the Serengeti Plains. Except, perhaps, the stage at the Y.W.C.A., where the Shadow Box Theater is celebrating Black History Month with ''The African Drum.'' Here, children can see a multimedia presentation of traditional African tales whose characters include a talking leopard, a wise loon and a turtle who is just as resourceful as the hero of ''The Tortoise and the Hare.'' All are shadow puppets, which are so deftly manipul

## SIMPLIFIED ARTICLES
Basically I have just included data from three field I considered are important for topic modelling

In [71]:
#basically creating a new_list which only has these details considered relevant by me for topic model
simplified_articles = [(str(article['description']) + str(article['text']) + str(article['title']) ) for article in articles_in_dic_format]

In [ ]:
print(simplified_articles[5])

In [79]:
import json

In [80]:
#writing this to a file for future use
with open('articles.json','w') as file:
    json.dump(simplified_articles,file)

In [81]:
#loading from file
with open('articles.json','r') as file:
    js_to_li= json.load(file)

## HEADOVER TO NOTEBOOK FOR LDA MODELLING